<a href="https://colab.research.google.com/github/dvlpana/SparkPlatzi/blob/main/CursoSpakPlatzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [298]:
#!apt-get install  openjdk-8-jdk-headless -qq > /dev/null
#!ad-apt-repository ppa:openjdk-r/ppa
#!apt-get update && sudo apt-get upgrade -y
#!apt-get -y install openjdk-8-jre
#!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
#!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
#!pip install -q findspark
#!pip install pyspark

In [299]:
!pip install -q findspark

In [300]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [301]:
import time
print("Sleeping")
time.sleep(30) # sleep for a while; interrupt me!
print("Done Sleeping")

Sleeping
Done Sleeping


In [302]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [303]:
!head -n 15 /content/sample_data/files_curso_spark/data.csv

State,Color,Count
TX,Red,20
NV,Blue,66
CO,Blue,79
OR,Blue,71
WA,Yellow,93
WY,Blue,16
CA,Yellow,53
WA,Green,60
OR,Green,71
TX,Green,68
NV,Green,59
AZ,Brown,95
WA,Yellow,20
AZ,Blue,75


In [304]:
!spark-submit /content/sample_data/files_curso_spark/codeExample.py /content/sample_data/files_curso_spark/data.csv

21/05/06 18:10:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/05/06 18:10:25 INFO SparkContext: Running Spark version 3.1.1
21/05/06 18:10:25 INFO ResourceUtils: ==============================================================
21/05/06 18:10:25 INFO ResourceUtils: No custom resources configured for spark.driver.
21/05/06 18:10:25 INFO ResourceUtils: ==============================================================
21/05/06 18:10:25 INFO SparkContext: Submitted application: PythonMnMCount
21/05/06 18:10:25 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/05/06 18:10:25 INFO Resour

In [305]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [306]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [307]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [308]:
archivo = './sample_data/files_curso_spark/data.csv'
df_spark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
print(type(df_spark))

<class 'pyspark.sql.dataframe.DataFrame'>


¿Nombre de las Columnas de dataframe?

In [309]:
df_spark.columns

['State', 'Color', 'Count']

Estructura del dataframe

In [310]:
df_spark.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



Ver los primeros 20 registros del dataframe

In [311]:
df_spark.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



Descricipcion Estadistica del dataframe

In [312]:
df_spark.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
State,99999,None,None,AZ,WY
Color,99999,None,None,Blue,Yellow
Count,99999,55.00090000900009,26.26648202084822,10,100


Descripcion estadistica de una sola columna (‘median_house_value’)

In [313]:
df_spark.describe(['color']).show()

+-------+------+
|summary| color|
+-------+------+
|  count| 99999|
|   mean|  null|
| stddev|  null|
|    min|  Blue|
|    max|Yellow|
+-------+------+



## 8 Transformaciones y Acciones

In [314]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")
sc



<SparkContext master=local[*] appName=pyspark-shell>

In [315]:
path = '/content/sample_data/files_curso_spark/'
equiposOlimpicosRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(","))

# Muestra la data con take , evita utilizar collect()

In [316]:
equiposOlimpicosRDD.take(10)



[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

9 Acciones de modificacion sobre RDDs

In [317]:
##Contar cuantas siglas tenemos
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [318]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(len).take(5)

[('ARG', 18), ('AFG', 1), ('IRL', 7), ('POR', 21), ('AUS', 23)]

In [319]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [320]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(list).take(3)

[('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan']),
 ('IRL',
  ['Akatonbo',
   'Aletta',
   'Ireland',
   'Ireland-1',
   'Ireland-2',
   'The Cloud',
   'Three Leaves'])]

In [321]:
##Obtener los datos sólo de un país o contenido con filter()
equiposArgentinos = equiposOlimpicosRDD.filter(lambda x: "ARG" in x)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [322]:
## Cuenta todos los valores, nada recomendable para datos extensos
equiposOlimpicosRDD.count()

1185

In [323]:
##Cuenta  hasta el tiempo aproximado
equiposOlimpicosRDD.countApprox(20)

1185

10 Acciones de conteo sobre RDDs

In [324]:
## importamos ambos
deportistaOlimpicoRDD = sc.textFile(path+'deportista.csv') \
.map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = sc.textFile(path+'deportista2.csv') \
.map(lambda line: line.split(","))

In [325]:
## Unimos los RDD con union(). Spark solo tiene esta operacion
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [326]:
deportistaOlimpicoRDD.count()

135572

In [327]:
##Ver contenido en equipos olimpicos
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [328]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [329]:
# seleccionamos las comumnas ID para hacer un join
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.take(6)

[('278', (['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0'], 'SWE')),
 ('278', (['86368', 'August Nilsson', '1', '27', '0', '0'], 'SWE')),
 ('278', (['112720', 'Gustaf Fredrik Sderstrm', '1', '34', '0', '0'], 'SWE')),
 ('278',
  (['114107', 'Karl Gustaf Vilhelm Staaf Johansson ', '1', '19', '0', '0'],
   'SWE')),
 ('278',
  (['130945', 'Charles Gustav Wilhelm Winckler', '1', '33', '0', '0'], 'SWE')),
 ('982', (['51', 'Nstor Abad Sanjun', '1', '23', '167', '64'], 'ESP'))]

In [330]:
# utilizamos takeSample (repetidos, muestra, semilla)
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.takeSample(False,6,25)

[('514', (['58198', 'Mayumi Kawasaki', '2', '24', '167', '52'], 'JPN')),
 ('399',
  (['129907', 'Armin Christian Weyrauch', '1', '28', '193', '95'], 'GER')),
 ('487', (['53275', 'Khashaba Dadasaheb Jahdav', '1', '21', '0', '0'], 'IND')),
 ('138',
  (['110517', 'Hlio Frederico Gomes da Silva', '1', '22', '183', '79'],
   'BRA')),
 ('507', (['121829', 'Nicola Trentin', '1', '30', '178', '70'], 'ITA')),
 ('138',
  (['23297', 'Kissya Cataldo da Costa', '2', '30', '180', '67'], 'BRA'))]

In [331]:
resultado = sc.textFile(path+'resultados.csv') \
.map(lambda line: line.split(","))

In [332]:
resultadoGanador = resultado.filter(lambda w: 'NA' not in w[1])

In [333]:
resultadoGanador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [334]:
##Reto Hacer un join con equipos y deportista para obtener valores importantes
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.join(resultadoGanador.map(lambda resultado: [resultado[2], resultado [1]])) \
.takeSample(False, 6, 25)


[('93',
  ((['73927', 'Yana Eduardovna Maksimova', '2', '19', '182', '70'], 'BLR'),
   'Gold')),
 ('705',
  ((['121086', 'Linda Toorop Moor ', '2', '16', '165', '53'], 'NED'), 'Gold')),
 ('1071',
  ((['118844', 'Nurcan Taylan', '2', '20', '151', '48'], 'TUR'), 'Silver')),
 ('705',
  ((['27539', 'Arnold Pieter Bernard Arno den Hartog', '1', '29', '187', '88'],
    'NED'),
   'Gold')),
 ('705',
  ((['66236', 'Frederik Joseph Frits Lamp', '1', '18', '0', '0'], 'NED'),
   'Bronze')),
 ('1181',
  ((['17015', 'Hastings Bwalya', '1', '23', '168', '64'], 'ZAM'), 'Gold'))]

Clase 11 Solucion reto deportistas

In [335]:
#esta clase muestra la solucion del profesor al reto.
deportistaPaises = deportistaOlimpicoRDD \
    .map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]]))

deportistaPaises.join(resultadoGanador).take(6)


[('972',
  ((['3585', 'An JaeHyeong', '1', '23', '183', '74'], 'KOR'), 'Silver')),
 ('972', ((['19788', 'Chang YeNa', '2', '26', '172', '61'], 'KOR'), 'Silver')),
 ('972',
  ((['40891', 'Go SeongHyeon', '1', '25', '182', '85'], 'KOR'), 'Silver')),
 ('972',
  ((['44419', 'Ha JeongEun', '2', '21', '173', '66'], 'KOR'), 'Silver')),
 ('972',
  ((['45476', 'Han SangHun', '1', '23', '175', '79'], 'KOR'), 'Silver')),
 ('972',
  ((['51378', 'Hwang HyeYeong', '2', '26', '0', '0'], 'KOR'), 'Silver'))]

#Clase 12 Operaciones Numéricas
##En esta clase hacemos el encoding de los valores de las medallas y obtenemos los puntos que tiene cada país.
Acciones de Conteo sobre RDD

```
# Tiene formato de código
```



In [336]:
!ls ./sample_data/files_curso_spark/

codeExample.py	deportista2.csv      evento.csv		    paises.csv
data.csv	deportista.csv	     juegos.csv		    resultados.csv
deporte.csv	deportistaError.csv  modelo_relacional.jpg


In [337]:
#Almacenamos el puntaje por medalla
valoresMedallas = {'Gold'  :7,
                   'Silver':5,
                   'Bronze':4}

In [338]:
paisesMedallas = deportistaPaises.join(resultadoGanador)



In [339]:
paisesMedallas = paisesMedallas \
    .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1] ] ) )

In [340]:
paisesMedallas.take(5)

[('KOR', 5), ('KOR', 5), ('KOR', 5), ('KOR', 5), ('KOR', 5)]

In [341]:
from operator import add

conclusion = paisesMedallas.reduceByKey((add)) \
    .sortBy(lambda x:x[1], ascending=False)

In [342]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

Clase 13 Creación de DataFrames

In [343]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext





In [344]:
#spark = SparkContext(master='local', appName="DataFrames")
sqlContext = SQLContext(spark)




In [345]:
!ls ./sample_data/files_curso_spark/

codeExample.py	deportista2.csv      evento.csv		    paises.csv
data.csv	deportista.csv	     juegos.csv		    resultados.csv
deporte.csv	deportistaError.csv  modelo_relacional.jpg


In [346]:
!head -n 10 ./sample_data/files_curso_spark/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina
5,1908 Verano,1908,Verano,London
6,1912 Verano,1912,Verano,Stockholm
7,1920 Verano,1920,Verano,Antwerpen
8,1924 Invierno,1924,Invierno,Chamonix
9,1924 Verano,1924,Verano,Paris


In [347]:
spark

In [348]:
# Paso 1 crear un dataFrame es crearun Schema como una base de datos
path = '/content/sample_data/files_curso_spark/'
juegoSchema = StructType([
     StructField('juego_id', IntegerType(), False),
     StructField('anio', StringType(), False),
     StructField('temporada', StringType(), False),
     StructField('ciudad', StringType(), False),
    ])
# Paso 2 Crea Data Frame con la Lectura del Schema 
juegoDF = sqlContext.read.schema(juegoSchema) \
          .option('header', 'true').csv(path+'resultados.csv')

In [349]:
juegoDF.show(4)

+--------+----+---------+------+
|juego_id|anio|temporada|ciudad|
+--------+----+---------+------+
|       1|  NA|        1|    39|
|       2|  NA|        2|    49|
|       3|  NA|        3|     7|
|       4|Gold|        4|     2|
+--------+----+---------+------+
only showing top 4 rows



Clase 14 Inferencia de tipos de datos

In [350]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [351]:
# Retiramos encabezados
def eliminarEncabezado(indice, iterador):
  return iter(list(iterador)[1:])

In [352]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
mapPartitionsWithIndex(eliminarEncabezado)

In [353]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [354]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6]),
))

In [355]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [380]:
schema_deportistaOlimpico = StructType([
    StructField('deportista_id', IntegerType(),False),
    StructField('nombre', StringType(),False),
    StructField('genero', IntegerType(),False),
    StructField('edad', IntegerType(),False),
    StructField('altura', IntegerType(),False),
    StructField('peso', FloatType(),False),
    StructField('equipo_id', IntegerType(),False)
]    
)

In [381]:
deportistaDF =  sqlContext.createDataFrame(deportistaOlimpicoRDD, schema_deportistaOlimpico)

In [382]:
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



Retos crear los demas DataFrames testantes a partir de los RDD

In [359]:
# Países  equipos, sc???
paisesRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(",")) \
.mapPartitionsWithIndex(eliminarEncabezado) \
.map(lambda l:(
    int(l[0]),
    l[1],
    l[2]
))

schema = StructType([
    StructField('pais_id', IntegerType(),False),
    StructField('equipo', StringType(),False),
    StructField('sigla', StringType(),False)

]    
)

paisesDF =  sqlContext.createDataFrame(paisesRDD, schema)
paisesDF.show(5)

+-------+--------------------+-----+
|pais_id|              equipo|sigla|
+-------+--------------------+-----+
|      1|         30. Februar|  AUT|
|      2|A North American ...|  MEX|
|      3|           Acipactli|  MEX|
|      4|             Acturus|  ARG|
|      5|         Afghanistan|  AFG|
+-------+--------------------+-----+
only showing top 5 rows



In [360]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])

resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")

resultadosDF.show(4)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
+------------+-------+-------------+--------+---------+
only showing top 4 rows



In [361]:
evento_schema = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("evento", StringType(), False), 
    StructField("deporte_id", IntegerType(), False)
])

eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

eventoDF.show(4)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
+---------+--------------------+----------+
only showing top 4 rows



In [362]:
deporte_schema = StructType([
          StructField("deporte_id",IntegerType(),False),
          StructField("deporte", StringType(),False)
])

deporteDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(deporte_schema).\
        load(path+"deporte.csv")

deporteDF.show(4)

+----------+----------+
|deporte_id|   deporte|
+----------+----------+
|         1|Basketball|
|         2|      Judo|
|         3|  Football|
|         4|Tug-Of-War|
+----------+----------+
only showing top 4 rows



In [363]:
# juegos

juegos_schema = StructType([
    StructField('nombre_juego', StringType(),False),
    StructField('annio', IntegerType(),False),
    StructField('temporada', StringType(),False),
    StructField('ciudad', StringType(),False)
]    
)

juegosDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(juegos_schema).\
        load(path+"juegos.csv")

juegosDF.show(4)


+------------+-----+---------+------+
|nombre_juego|annio|temporada|ciudad|
+------------+-----+---------+------+
|           1| null|     1896|Verano|
|           2| null|     1900|Verano|
|           3| null|     1904|Verano|
|           4| null|     1906|Verano|
+------------+-----+---------+------+
only showing top 4 rows



 Clase 15 Operaciones sobre DF.
En esta clase aprendemos como obtener el schema de un DataFrame, a renombrar columnas y select().

In [364]:
# Ver el esquema
deporteDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [365]:
# Ver el esquema
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [366]:
#Renombrar columnas
deportistaOlimpicoDF = deportistaDF \
.withColumnRenamed("genero", 'sexo') \
.drop('altura')


In [370]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- edadAlJugar: integer (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [378]:
# Usando el rey SELECT
from pyspark.sql.functions import *
deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id","nombre",
 col("edad").alias("edadAlJugar"),
 "equipo_id"
 )

AnalysisException: ignored

In [379]:
deportistaOlimpicoDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [375]:
# Usando filter
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(
    (deportistaOlimpicoDF.edadAlJugar != 0)
)

In [377]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|       118925|Megan Olwen Deven...|         11|      413|
|        76675|   Marcelle Matthews|         11|      967|
|        70616|          Liu Luyang|         11|      199|
|        47618|Sonja Henie Toppi...|         11|      742|
|        37333|Carlos Bienvenido...|         11|      982|
|        51268|      Beatrice Hutiu|         11|      861|
|        22411|Magdalena Cecilia...|         11|      413|
|        40129|    Luigina Giavotti|         11|      507|
|        52070|        Etsuko Inada|         11|      514|
|       126307|        Liana Vicens|         11|      825|
|        43528|Antoinette Joyce ...|         12|      172|
|        46578|        Diana Hatler|         12|      825|
|        42835|   Werner Grieshofer|         12|       7